<a href="https://colab.research.google.com/github/Pamela2710/Data-Mining/blob/main/DataProcessingClassification/DataProcessing_and_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel, mutual_info_classif
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.base import clone
from scipy.stats import entropy
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest

df = pd.read_csv('https://raw.githubusercontent.com/Pamela2710/Data-Mining/main/DataProcessing/features_completas.csv', encoding='latin-1')

# Eliminar filas con valores NaN
df = df.dropna()

df

,Media tiempo,Desviacion estandar tiempo,Varianza tiempo,Entropia tiempo,kurtosis tiempo,MSE tiempo,Tiempo en alcanzar el pico maximo en frecuencia,RMS tiempo,Peak2peak tiempo,TIEMPO peak2rms,...,solidity,std_dev,x_center_mass,x_centroid,x_centroid_cm_diff,y_center_mass,y_centroid,y_centroid_cm_diff,Etiqueta,espectrograma etiqueta
0,0.099520,0.245902,0.108890,0,0.013447,0.907095,0.115646,0.245920,0.164061,0.110557,...,1,0.410795,0.406519,0.405904,-0.000615,0.472183,0.471469,-0.000713,Long Period,espect_event_1
1,0.085819,0.876726,0.796870,0,0.050793,0.351758,0.306122,0.876697,1.000000,0.238391,...,1,12.640444,0.240467,0.239852,-0.000615,0.279601,0.278887,-0.000713,Volcano Tectonic,espect_event_2
2,0.077419,0.231336,0.099950,0,0.015484,0.882481,0.122449,0.231357,0.154656,0.130988,...,1,4.451864,0.422509,0.421894,-0.000615,0.490728,0.490014,-0.000713,Long Period,espect_event_3
3,0.102426,0.291445,0.138865,0,0.011577,0.910015,0.102041,0.291448,0.160484,0.092834,...,1,7.681243,0.418819,0.418204,-0.000615,0.486448,0.485735,-0.000713,Long Period,espect_event_4
4,0.091737,0.428982,0.247991,0,0.026587,0.723575,0.108844,0.428999,0.379571,0.141011,...,1,2.908373,0.287208,0.286593,-0.000615,0.333809,0.333096,-0.000713,Hybrid,espect_event_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,0.101867,0.214477,0.089995,0,0.018980,0.875568,0.333333,0.214480,0.163796,0.131218,...,1,2.546444,0.421894,0.421279,-0.000615,0.490014,0.489301,-0.000713,Long Period,espect_event_686
686,0.086898,0.680620,0.520007,0,0.038755,0.298080,0.455782,0.680617,0.504337,0.150002,...,1,15.365171,0.325953,0.325338,-0.000615,0.378745,0.378031,-0.000713,Volcano Tectonic,espect_event_687
687,0.077123,0.548769,0.365809,0,0.024871,0.207060,0.510204,0.548773,0.259238,0.084483,...,1,9.442264,0.306888,0.306273,-0.000615,0.356633,0.355920,-0.000713,Volcano Tectonic,espect_event_688
688,0.079244,0.720172,0.571271,0,0.031290,0.165912,0.551020,0.720172,0.407933,0.102759,...,1,14.023466,0.337638,0.337023,-0.000615,0.392297,0.391583,-0.000713,Volcano Tectonic,espect_event_689


In [17]:
# Extraer la columna "Etiqueta"
etiquetas = df['Etiqueta']

# Normalizar las columnas numéricas
cols_numericas = df.columns[:-3]  # Excluyendo las columnas "Etiqueta" y "espectrograma etiqueta"
scaler = MinMaxScaler()
df[cols_numericas] = scaler.fit_transform(df[cols_numericas])

# Unir la columna "Etiqueta" a los datos normalizados
df_normalized = pd.concat([df[cols_numericas], etiquetas], axis=1)

# Mostrar el resultado
df_normalized

,Media tiempo,Desviacion estandar tiempo,Varianza tiempo,Entropia tiempo,kurtosis tiempo,MSE tiempo,Tiempo en alcanzar el pico maximo en frecuencia,RMS tiempo,Peak2peak tiempo,TIEMPO peak2rms,...,perimeter,skewness,solidity,std_dev,x_center_mass,x_centroid,x_centroid_cm_diff,y_center_mass,y_centroid,Etiqueta
0,0.099520,0.226593,0.102626,0.0,0.013447,0.907095,0.115646,0.226602,0.163998,0.110557,...,0.750000,0.262347,0.0,0.014767,0.503906,0.503906,0.0,0.503906,0.503906,Long Period
1,0.085819,0.873569,0.795442,0.0,0.050793,0.351758,0.306122,0.873538,1.000000,0.238391,...,0.402174,0.009667,0.0,0.499546,0.152344,0.152344,0.0,0.152344,0.152344,Volcano Tectonic
2,0.077419,0.211654,0.093624,0.0,0.015484,0.882481,0.122449,0.211665,0.154592,0.130988,...,0.827640,0.046256,0.0,0.174953,0.537760,0.537760,0.0,0.537760,0.537760,Long Period
3,0.102426,0.273302,0.132812,0.0,0.011577,0.910015,0.102041,0.273296,0.160420,0.092834,...,0.874224,0.024948,0.0,0.302965,0.529948,0.529948,0.0,0.529948,0.529948,Long Period
4,0.091737,0.414361,0.242705,0.0,0.026587,0.723575,0.108844,0.414371,0.379524,0.141011,...,0.647516,0.071298,0.0,0.113770,0.251302,0.251302,0.0,0.251302,0.251302,Hybrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,0.101867,0.194363,0.083598,0.0,0.018980,0.875568,0.333333,0.194357,0.163733,0.131218,...,0.354037,0.062165,0.0,0.099423,0.536458,0.536458,0.0,0.536458,0.536458,Long Period
686,0.086898,0.672442,0.516633,0.0,0.038755,0.298080,0.455782,0.672435,0.504299,0.150002,...,0.583851,0.012588,0.0,0.607553,0.333333,0.333333,0.0,0.333333,0.333333,Volcano Tectonic
687,0.077123,0.537215,0.361351,0.0,0.024871,0.207060,0.510204,0.537213,0.259182,0.084483,...,0.591615,0.020329,0.0,0.372771,0.292969,0.292969,0.0,0.292969,0.292969,Volcano Tectonic
688,0.079244,0.713007,0.568258,0.0,0.031290,0.165912,0.551020,0.713004,0.407889,0.102759,...,0.482919,0.014551,0.0,0.554368,0.358073,0.358073,0.0,0.358073,0.358073,Volcano Tectonic


In [59]:
import numpy as np

def objective_function(selected_features, X_train, Y_train):
    # Convert binary vector to feature indices
    selected_indices = np.where(selected_features == 1)[0]
    print('selected_indices! ', selected_indices)
    # Calculate Gain Ratio for the selected features
    info_gain = mutual_info_classif(X_train[:, selected_indices], Y_train, discrete_features='auto')
    print('info_gain ', info_gain)
    intrinsic_info = [entropy(X_train[:, i]) for i in selected_indices]


    # Cambiar valores NaN a 0
    intrinsic_info_sin_nan = np.nan_to_num(intrinsic_info)
    print('intrinsic_info type', type(intrinsic_info_sin_nan))
    print('intrinsic_info ', intrinsic_info_sin_nan)


    # Calculate Gain Ratio as the fitness score
    gain_ratio = [ig / ii if (ii > 0 and ig > 0)  else 0 for ig, ii in zip(info_gain, intrinsic_info_sin_nan)]
    print('gain ', gain_ratio)

    # Sum the Gain Ratios to get the overall fitness score (maximize Gain Ratio)
    fitness = sum(gain_ratio)


    return -fitness  # Minimize the negative fitness (maximize Gain Ratio)

def artificial_bee_colony(X_train, Y_train, num_iterations, num_employed_bees, num_features):
    results = []

    for _ in range(num_iterations):
        # Randomly generate employed bees' solutions
        solutions = np.random.choice([0, 1], size=(num_employed_bees, num_features))
        print('soluciones', solutions)

        # Evaluate the solutions
        scores = np.zeros(num_employed_bees)
        for i in range(num_employed_bees):
            print('soluciones i', solutions[i])
            score = objective_function(solutions[i], X_train, Y_train)  # Utiliza la función objetivo adecuada
            scores[i] = score
            print('scores i', scores[i])
            # Guarda los resultados
            results.append((solutions[i], score))

        # Update solutions based on scores
        for i in range(num_employed_bees):
            if np.random.rand() < scores[i] / np.max(scores): ####################################
                solutions[i] = np.random.choice([0, 1], size=num_features)
                print('solutions2 i', solutions[i])


    # Ordena los resultados por importancia
    results.sort(key=lambda x: x[1], reverse=True)

    return results[:5]  # Devuelve los cinco mejores subconjuntos

X_normalized = scaler.fit_transform(df_normalized.drop(['Etiqueta'], axis=1))
y = df_normalized['Etiqueta']

# 'X_normalized' es el conjunto de datos normalizado y 'y' son las etiquetas
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

num_features = X_train.shape[1]
num_iterations = 60
num_employed_bees = 20

# Ejecuta el algoritmo ABC
best_subsets = artificial_bee_colony(X_train, y_train, num_iterations, num_employed_bees, num_features)

# Imprime los mejores subconjuntos
for i, subset in enumerate(best_subsets):
    print(f"Mejor subconjunto {i+1}: {subset}")


soluciones [[0 0 0 ... 0 1 1]
 [1 0 0 ... 0 0 1]
 [0 1 0 ... 0 1 1]
 ...
 [0 1 0 ... 0 1 0]
 [0 1 1 ... 0 1 0]
 [1 0 0 ... 0 1 1]]
soluciones i [0 0 0 0 1 1 0 0 1 1 1 1 0 0 1 0 1 0 1 1 0 0 1 0 1 0 0 1 1 0 1 1 1 0 0 0 0
 0 1 1 0 0 0 0 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1
 0 0 1 1 0 1 1 1 1 0 1 0 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 0 0 1 0 1 1]
selected_indices!  [  4   5   8   9  10  11  14  16  18  19  22  24  27  28  30  31  32  38
  39  44  45  46  47  48  50  52  53  54  55  60  63  68  71  73  76  77
  79  80  81  82  84  87  88  90  91  93  94  96  97  99 100 104 106 107]
info_gain  [0.04119779 0.10987455 0.05296256 0.07438085 0.05724899 0.00605559
 0.09884721 0.03298463 0.         0.05767561 0.1547914  0.0920809
 0.14315327 0.02437054 0.03108626 0.13610195 0.00565886 0.02195238
 0.01239987 0.07626141 0.07040521 0.07506347 0.07713982 0.11226809
 0.01688153 0.00056251 0.04413973 0.0538392  0.01566017 0.02502232
 0.01277833 0.04727568 0.01363273 0.         0.04044

/usr/local/lib/python3.10/dist-packages/scipy/stats/_entropy.py:133: RuntimeWarning: divide by zero encountered in divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_entropy.py:133: RuntimeWarning: invalid value encountered in divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)


info_gain  [0.0155679  0.00840604 0.11111699 0.06581288 0.01042182 0.05297085
 0.05769607 0.00506628 0.06787479 0.07451582 0.00441587 0.
 0.         0.05764346 0.07414435 0.024361   0.13398183 0.03107399
 0.12984588 0.         0.0186238  0.0131611  0.04555822 0.07040832
 0.07506347 0.18872482 0.18757129 0.000589   0.04414866 0.05416255
 0.01566482 0.1122675  0.01275313 0.01041646 0.         0.
 0.01365011 0.13637113 0.04044398 0.19520787 0.         0.03198812
 0.01380915 0.14617449 0.01836972 0.         0.03161976 0.15793914
 0.00762574 0.01983045 0.02238392 0.03640168 0.18757388 0.18423413]
intrinsic_info type <class 'numpy.ndarray'>
intrinsic_info  [6.2435045  0.         6.27324628 6.09111364 6.15072022 6.15508195
 5.97668099 6.16824446 6.26173028 6.09111364 0.         5.87293495
 4.91786783 5.97668099 6.15846081 5.9806997  6.11496489 6.14569205
 5.73098145 5.71280951 6.25420413 6.0815922  6.09896583 6.16926195
 6.0827588  5.84522182 6.00824659 6.16593663 6.18266101 5.9801653
 5.8235

/usr/local/lib/python3.10/dist-packages/scipy/stats/_entropy.py:133: RuntimeWarning: divide by zero encountered in divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_entropy.py:133: RuntimeWarning: invalid value encountered in divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)


info_gain  [0.01076983 0.0412097  0.10988925 0.07785693 0.05235743 0.05722852
 0.0044749  0.0878702  0.02854376 0.03298463 0.         0.
 0.05809875 0.15198223 0.09346212 0.05261476 0.07412391 0.03106779
 0.00566494 0.01711445 0.01257713 0.01238091 0.01313955 0.
 0.04559073 0.07575292 0.07041817 0.11225401 0.18874022 0.02472216
 0.1875835  0.0005575  0.01629039 0.08727405 0.1532453  0.01676858
 0.03841873 0.         0.04724031 0.         0.0955369  0.1363709
 0.19459167 0.11415787 0.         0.12009943 0.0311002  0.01379233
 0.15192897 0.15031577 0.01762619 0.         0.03039318 0.15789145
 0.00765079 0.01077176 0.02240001 0.00532379 0.03131232 0.18961104
 0.18792811 0.1888791  0.18768976]
intrinsic_info type <class 'numpy.ndarray'>
intrinsic_info  [6.15072023 5.93266894 6.27324628 6.09111364 6.15508195 5.97668099
 6.16824446 5.95844615 0.         5.97928215 5.87293495 4.91786783
 6.07113196 5.8203237  5.83302307 5.94901178 6.15846081 6.14569205
 6.15234318 6.05375078 6.25420413 6.0932

...

info_gain  [0.01739764 0.01076983 0.010468   0.04120432 0.06597152 0.00623704
 0.10262839 0.02080064 0.         0.03455008 0.03106338 0.00566494
 0.         0.02762924 0.01652421 0.01238091 0.01313283 0.04555029
 0.00341787 0.07738421 0.07506347 0.18872482 0.18760212 0.00056251
 0.05420547 0.14431634 0.08727499 0.11228752 0.15323453 0.01429162
 0.01276322 0.0383398  0.01041696 0.         0.01987673 0.
 0.01364126 0.19459167 0.11414716 0.         0.11885163 0.03198812
 0.01379591 0.1511311  0.15705861 0.         0.15326669 0.03161602
 0.15472077 0.03583834 0.01153819 0.0312967  0.18102858 0.18420433]
intrinsic_info type <class 'numpy.ndarray'>
intrinsic_info  [6.2435045  6.15072023 6.00778055 5.93266894 6.09111364 6.16824446
 5.95844615 0.         4.91786783 5.85291356 6.14569205 6.15234318
 5.71280951 6.09265107 6.05375078 6.09328088 6.0815922  6.09896583
 6.14493777 5.99737027 6.0827588  5.84522182 6.00824659 6.16593663
 5.9801653  6.0055784  6.20162015 5.95466779 5.57122903 6.0573529

/usr/local/lib/python3.10/dist-packages/scipy/stats/_entropy.py:133: RuntimeWarning: divide by zero encountered in divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_entropy.py:133: RuntimeWarning: invalid value encountered in divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)


info_gain  [1.73981535e-02 1.07491853e-02 1.14060095e-02 4.03904505e-03
 4.11852277e-02 1.09889245e-01 7.74852811e-02 1.13262548e-02
 7.43742989e-02 1.07091328e-01 8.43380774e-03 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.81227662e-02 9.09807024e-02
 2.43705440e-02 1.30256964e-01 3.10862597e-02 1.44136256e-01
 7.89848096e-02 3.03556250e-02 1.70378154e-02 1.23809075e-02
 0.00000000e+00 9.19856717e-02 7.04140606e-02 1.88744811e-01
 2.26992779e-02 1.35391529e-04 1.12311522e-01 1.67427576e-02
 1.27682252e-02 3.83188691e-02 3.18128832e-02 1.04122745e-02
 0.00000000e+00 2.04769616e-02 9.55406856e-02 1.88734643e-01
 1.14164528e-01 0.00000000e+00 1.20099154e-01 1.38131793e-02
 1.50581833e-01 1.53999025e-01 1.56422936e-01 8.16456086e-03
 0.00000000e+00 3.64063743e-02 0.00000000e+00 1.09089611e-02
 1.87673085e-01 1.85203865e-01]
intrinsic_info type <class 'numpy.ndarray'>
intrinsic_info  [6.2435045  6.15072023 6.00778055 0.         5.93266894 6.27324628
 6.09111364 6.15072022 6.22343201 5.9

/usr/local/lib/python3.10/dist-packages/scipy/stats/_entropy.py:133: RuntimeWarning: divide by zero encountered in divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)
/usr/local/lib/python3.10/dist-packages/scipy/stats/_entropy.py:133: RuntimeWarning: invalid value encountered in divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)


KeyboardInterrupt: 